In [ ]:
print("good mornning")

good mornning


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import tensorflow as tf


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


<h2>Load the Data</h2>

In [ ]:
# Define file paths for each dataset
ratings_file = 'ratings.csv'
tags_file = 'tags.csv'
movies_file = 'movies.csv'
links_file = 'links.csv'

# Load data into Pandas DataFrames
ratings_data = pd.read_csv(ratings_file)
tags_data = pd.read_csv(tags_file)
movies_data = pd.read_csv(movies_file)
links_data = pd.read_csv(links_file)

# Display the first few rows of each dataset for verification
print("Ratings Data:")
print(ratings_data.head())

print("\nTags Data:")
print(tags_data.head())

print("\nMovies Data:")
print(movies_data.head())

print("\nLinks Data:")
print(links_data.head())

Ratings Data:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Tags Data:
   userId  movieId              tag   timestamp
0       2    60756            funny  1445714994
1       2    60756  Highly quotable  1445714996
2       2    60756     will ferrell  1445714992
3       2    89774     Boxing story  1445715207
4       2    89774              MMA  1445715200

Movies Data:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantas

In [ ]:
# Load the dataset (replace with your file paths)
ratings_data = pd.read_csv('ratings.csv')
tags_data = pd.read_csv('tags.csv')
movies_data = pd.read_csv('movies.csv')

# Explore the Ratings Data
print("Ratings Data Exploration:")
print("Number of Ratings:", len(ratings_data))
print("Unique Users:", ratings_data['userId'].nunique())
print("Unique Movies:", ratings_data['movieId'].nunique())
print("Ratings Distribution:")
print(ratings_data['rating'].value_counts().sort_index())

# Explore the Tags Data
print("\nTags Data Exploration:")
print("Number of Tags:", len(tags_data))
print("Unique Users:", tags_data['userId'].nunique())
print("Unique Movies:", tags_data['movieId'].nunique())
print("Unique Tags:", tags_data['tag'].nunique())

# Explore the Movies Data
print("\nMovies Data Exploration:")
print("Number of Movies:", len(movies_data))
print("Movie Genres:")
genre_counts = movies_data['genres'].str.split('|').explode().value_counts()
print(genre_counts)

# Additional Exploration (if needed)
# - Check for missing values
# - Explore timestamp information
# - Visualize data distributions (e.g., histograms, bar plots)

# Sample visualization (requires matplotlib)
# import matplotlib.pyplot as plt
# ratings_data['rating'].hist(bins=9, grid=False)
# plt.xlabel('Rating')
# plt.ylabel('Frequency')
# plt.title('Distribution of Ratings')
# plt.show()


Ratings Data Exploration:
Number of Ratings: 100836
Unique Users: 610
Unique Movies: 9724
Ratings Distribution:
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: rating, dtype: int64

Tags Data Exploration:
Number of Tags: 3683
Unique Users: 58
Unique Movies: 1572
Unique Tags: 1589

Movies Data Exploration:
Number of Movies: 9742
Movie Genres:
Drama                 4361
Comedy                3756
Thriller              1894
Action                1828
Romance               1596
Adventure             1263
Crime                 1199
Sci-Fi                 980
Horror                 978
Fantasy                779
Children               664
Animation              611
Mystery                573
Documentary            440
War                    382
Musical                334
Western                167
IMAX                   158
Film-Noir               87
(no genres listed)      34
Name: genres, dtype: int64


In [ ]:
# Data Preprocessing for Ratings Data
# 1. Handle Missing Values (if any)
ratings_data.dropna(inplace=True)

# 2. Convert Timestamp to Datetime (if needed)
ratings_data['timestamp'] = pd.to_datetime(ratings_data['timestamp'], unit='s')

# Data Preprocessing for Tags Data
# 1. Handle Missing Values (if any)
tags_data.dropna(inplace=True)

# Data Preprocessing for Movies Data
# 1. Extract Release Year from Title
movies_data['year'] = movies_data['title'].str.extract(r'\((\d{4})\)')
movies_data['year'] = pd.to_numeric(movies_data['year'], errors='coerce')

# 2. Extract Movie Genres into Binary Columns
genres = movies_data['genres'].str.split('|')
genres = genres.apply(lambda x: '|'.join(sorted(x)))
movies_data = pd.concat([movies_data, genres.str.get_dummies()], axis=1)

# Sample Data After Preprocessing
print("Sample Data After Preprocessing:")
print("Ratings Data:")
print(ratings_data.head())
print("\nTags Data:")
print(tags_data.head())
print("\nMovies Data:")
print(movies_data.head())

Sample Data After Preprocessing:
Ratings Data:
   userId  movieId  rating           timestamp
0       1        1     4.0 2000-07-30 18:45:03
1       1        3     4.0 2000-07-30 18:20:47
2       1        6     4.0 2000-07-30 18:37:04
3       1       47     5.0 2000-07-30 19:03:35
4       1       50     5.0 2000-07-30 18:48:51

Tags Data:
   userId  movieId              tag   timestamp
0       2    60756            funny  1445714994
1       2    60756  Highly quotable  1445714996
2       2    60756     will ferrell  1445714992
3       2    89774     Boxing story  1445715207
4       2    89774              MMA  1445715200

Movies Data:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres    year  